In [171]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd 
#url to get the table of additives
url = "https://www.cspinet.org/api/tables/15"

response = requests.get(url)

if response.status_code != 200:
    print(f"Failed to fetch the page. Status code: {response.status_code}")
    exit()
data = response.json()# data is in json format 
#print(data)
additives = data['table_data']
#print(data['table_data'])

avoid_additives =[entry for entry in  data['table_data'] if entry['cell_1']=='Avoid']
    #url to get the health concerns for the addictives
base_url = 'https://www.cspinet.org/article/'
    #print(avoid_additives)
results = []
for entry in avoid_additives:
    name = entry['cell_0']
    purpose =entry['cell_2']
    #health_concern = 'N/A'
    url_hc = base_url+name.lower().replace(" ","-")
    detail_response=requests.get(url_hc)
    detail_soup = BeautifulSoup(detail_response.content,'html.parser')
    if detail_response.status_code == 200:
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        p_tags = detail_soup.find_all('p')
        for p in p_tags:
            strong_tags = p.find('strong')
            if strong_tags and 'Health Concerns:' in strong_tags.get_text():
                health_concerns_p = p
                health_concern =p.get_text().replace('Health Concerns:','').strip()
                #print(health_concerns)
                break;
            #else:
             #   print(f"Failed to fetch detail page for {name}. Status code: {detail_response.status_code}")
    else:
        health_concern=entry['cell_3']
    results.append({
            'ingredient':name,
            'purpose':purpose,
            'health concern':health_concern
        })
    
    # Print the latest result
    #print(results[-1])

    # Add a delay to be respectful to the server
    time.sleep(1)

df = pd.DataFrame(results)
df.to_csv('./data/avoid_additives.csv',index=False)
            
        
      